<a href="https://colab.research.google.com/github/ahmedhamodien/Machine-Learning-/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Github clone

In [1]:
!git clone https://github.com/your_username/Machine_Learning.git
%cd face-mask-detection


Cloning into 'Machine_Learning'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'face-mask-detection'
/content


# Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image

from tensorflow.keras.applications.densenet import preprocess_input

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import ViTFeatureExtractor, ViTForImageClassification
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tqdm.keras import TqdmCallback

# Data preprocessing

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ahmedhamodien","key":"5753d07a0e9a5fb93f192eec273677a3"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset


Dataset URL: https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset
License(s): CC0-1.0
 91% 301M/330M [00:00<00:00, 664MB/s]
100% 330M/330M [00:00<00:00, 682MB/s]


In [5]:
import zipfile

zip_ref = zipfile.ZipFile("face-mask-12k-images-dataset.zip", 'r')
zip_ref.extractall("face_mask_dataset_after_extraction")
zip_ref.close()


In [7]:
train_dir = '/content/face_mask_dataset_after_extraction'
train_dir

'/content/face_mask_dataset_after_extraction'

In [8]:
os.path.exists(train_dir)


True

In [9]:
import os
print(os.listdir("/content"))


['.config', 'face_mask_dataset_after_extraction', 'face-mask-12k-images-dataset.zip', 'kaggle.json', 'sample_data']


In [10]:
for root, dirs, files in os.walk("/content"):
    print(f"📁 {root}")
    for d in dirs:
        print(f"  📂 {d}")
    for f in files[:5]:  # Only show first 5 files per folder
        print(f"  🖼️ {f}")


📁 /content
  📂 .config
  📂 face_mask_dataset_after_extraction
  📂 sample_data
  🖼️ face-mask-12k-images-dataset.zip
  🖼️ kaggle.json
📁 /content/.config
  📂 logs
  📂 configurations
  🖼️ default_configs.db
  🖼️ .last_opt_in_prompt.yaml
  🖼️ config_sentinel
  🖼️ hidden_gcloud_config_universe_descriptor_data_cache_configs.db
  🖼️ active_config
📁 /content/.config/logs
  📂 2025.07.29
📁 /content/.config/logs/2025.07.29
  🖼️ 13.36.15.785307.log
  🖼️ 13.36.36.776594.log
  🖼️ 13.36.37.567476.log
  🖼️ 13.36.27.119914.log
  🖼️ 13.35.49.066514.log
📁 /content/.config/configurations
  🖼️ config_default
📁 /content/face_mask_dataset_after_extraction
  📂 Face Mask Dataset
📁 /content/face_mask_dataset_after_extraction/Face Mask Dataset
  📂 Validation
  📂 Train
  📂 Test
📁 /content/face_mask_dataset_after_extraction/Face Mask Dataset/Validation
  📂 WithMask
  📂 WithoutMask
📁 /content/face_mask_dataset_after_extraction/Face Mask Dataset/Validation/WithMask
  🖼️ Augmented_646_7625934.png
  🖼️ Augmented_223_6

In [11]:
import os


classes = sorted(os.listdir(train_dir))

print("Detected classes:", classes)
import os

# List files in root
print(os.listdir("/content"))
print(os.listdir("/content/face_mask_dataset_after_extraction/Face Mask Dataset/Train"))



Detected classes: ['Face Mask Dataset']
['.config', 'face_mask_dataset_after_extraction', 'face-mask-12k-images-dataset.zip', 'kaggle.json', 'sample_data']
['WithMask', 'WithoutMask']


In [12]:
train_dir = "face_mask_dataset_after_extraction/Face Mask Dataset/Train"


In [13]:
import os
import pandas as pd

# Change this to your actual folder on Colab after uploadiface_mask_dataset_after_extraction/Face Mask Datasetng from Kaggle

filepaths = []
labels = []

for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        for file in os.listdir(class_path):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                filepath = os.path.join(class_path, file)
                filepaths.append(filepath)
                labels.append(class_name)

import pandas as pd
train_df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})
print(train_df.head())
print("Total images found:", len(train_df))

                                           filepaths    labels
0  face_mask_dataset_after_extraction/Face Mask D...  WithMask
1  face_mask_dataset_after_extraction/Face Mask D...  WithMask
2  face_mask_dataset_after_extraction/Face Mask D...  WithMask
3  face_mask_dataset_after_extraction/Face Mask D...  WithMask
4  face_mask_dataset_after_extraction/Face Mask D...  WithMask
Total images found: 10000


In [14]:
print(filepaths[0])
print(os.path.exists(filepaths[0]))  # Should be True


face_mask_dataset_after_extraction/Face Mask Dataset/Train/WithMask/Augmented_374_1747794.png
True


In [15]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_df, test_size=0.2, stratify=train_df['labels'], random_state=42)

print("Train size:", len(train_data))
print("Validation size:", len(val_data))


Train size: 8000
Validation size: 2000


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image generators
train_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=20,
                               zoom_range=0.2,
                               horizontal_flip=True)

val_gen = ImageDataGenerator(rescale=1./255)

# Flow from dataframe
train_flow = train_gen.flow_from_dataframe(train_data,
                                           x_col='filepaths',
                                           y_col='labels',
                                           target_size=(224, 224),
                                           class_mode='binary',
                                           batch_size=32,
                                           shuffle=True)

val_flow = val_gen.flow_from_dataframe(val_data,
                                       x_col='filepaths',
                                       y_col='labels',
                                       target_size=(224, 224),
                                       class_mode='binary',
                                       batch_size=32,
                                       shuffle=True)


Found 8000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # ⬅️ 1 output neuron for binary
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
print(train_df['labels'].unique())
print(train_df['labels'].nunique())


['WithMask' 'WithoutMask']
2


In [19]:
print(len(train_data))
print(train_data.head())
import os

# Test one image path
print(filepaths[0])
print(os.path.exists(filepaths[0]))


8000
                                              filepaths       labels
2858  face_mask_dataset_after_extraction/Face Mask D...     WithMask
6842  face_mask_dataset_after_extraction/Face Mask D...  WithoutMask
3905  face_mask_dataset_after_extraction/Face Mask D...     WithMask
7684  face_mask_dataset_after_extraction/Face Mask D...  WithoutMask
1564  face_mask_dataset_after_extraction/Face Mask D...     WithMask
face_mask_dataset_after_extraction/Face Mask Dataset/Train/WithMask/Augmented_374_1747794.png
True


In [20]:
print(train_data.shape)
print(val_data.shape)

(8000, 2)
(2000, 2)


In [21]:
print("Train samples:", train_flow.samples)
print("Val samples:", val_flow.samples)


Train samples: 8000
Val samples: 2000


In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['encoded_labels'] = le.fit_transform(train_df['labels'])  # e.g., 'with_mask' → 1, 'without_mask' → 0


In [23]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(train_df['encoded_labels'])  # or use pd.get_dummies


In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(train_flow,
                    validation_data=val_flow,
                    epochs=10)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


133/250 ━━━━━━━━━━━━━━━━━━━━ 46s 396ms/step - accuracy: 0.5257 - loss: 0.0000e+00

KeyboardInterrupt: 